In [17]:
import geopandas as gpd, numpy as np
import pandas as pd
import rasterio as rio
from shapely.geometry import LineString, Point

In [19]:
# Carica il GeoDataFrame lineare
gdf = gpd.read_file('C:\\Users\\niett\\Downloads\\shp_linea.shp')  # Assicurati di usare il tuo percorso corretto al file

# Carica il DEM
with rio.open('DEM/dem_resample.tif') as src:  # Anche qui usa il tuo percorso al DEM
    affine = src.transform
    elevation = src.read(1)  # Assumi che il DEM abbia una sola banda

def get_elevation(x, y):
    row, col = rio.transform.rowcol(affine, x, y)
    return elevation[row, col]

def interpolate_points(line, interval):
    # Calcola il numero di segmenti in base alla lunghezza della linea e all'intervallo
    length = line.length
    
    # Interpola i punti lungo la linea
    points = [line.interpolate(distance) for distance in np.arange(0, length, interval)]
    if not line.is_closed:
        last_point = line.boundary.geoms[-1]  # Ottiene l'ultimo punto
        points.append(last_point)
    return points

def extract_segments_and_elevations(line, pixel_size):
    interpolated_points = interpolate_points(line, pixel_size)
    elevations = [get_elevation(p.x, p.y) for p in interpolated_points]
    elevation_differences = [j - i for i, j in zip(elevations[:-1], elevations[1:])]
    
    # Crea segmenti e associa le differenze di elevazione
    segments = [LineString([interpolated_points[i], interpolated_points[i+1]]) for i in range(len(interpolated_points) - 1)]
    
    return pd.DataFrame({'geometry': segments, 'elevation_difference': elevation_differences})

# Calcola la dimensione del pixel dal DEM
pixel_size = src.res[0]  # Assumi che il raster abbia la stessa risoluzione in x e y

# Applica la funzione a ogni geometria nel GeoDataFrame
segments_list = gdf['geometry'].apply(lambda geom: extract_segments_and_elevations(geom, pixel_size))

# Concatena tutti i DataFrames in uno solo
segments_df = pd.concat(segments_list.tolist()).reset_index(drop=True)

# Crea un GeoDataFrame dai segmenti
segments_gdf = gpd.GeoDataFrame(segments_df, geometry='geometry')
segments_gdf.crs = gdf.crs  # Assicurati di assegnare il sistema di riferimento corretto

In [20]:
segments_gdf

,geometry,elevation_difference
0,"LINESTRING (564732.318 4928435.168, 564937.715...",32.419861
1,"LINESTRING (564937.715 4928577.688, 565143.112...",-93.663788
2,"LINESTRING (565143.112 4928720.209, 565348.509...",48.302917
3,"LINESTRING (565348.509 4928862.729, 565553.906...",-45.006104
4,"LINESTRING (565553.906 4929005.249, 565759.303...",-1.873901
...,...,...
1134,"LINESTRING (750784.533 4891275.451, 750897.574...",46.122627
1135,"LINESTRING (750897.574 4891052.467, 751010.614...",13.971512
1136,"LINESTRING (751010.614 4890829.483, 751123.655...",-33.122108
1137,"LINESTRING (751123.655 4890606.499, 751236.695...",31.066200


In [21]:
segments_gdf.to_file('linea.shp')

C:\Users\niett\AppData\Local\Temp\ipykernel_13620\3779152106.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  segments_gdf.to_file('linea.shp')


In [22]:
# Carica il raster della velocità del vento
wind_raster_path = 'Storico meteo/Raster_export/average_normalized_raster.tif'
with rio.open(wind_raster_path) as wind_src:
    wind_affine = wind_src.transform
    wind_speed = wind_src.read(1)

# Funzione per ottenere la velocità del vento
def get_wind_speed(x, y):
    row, col = rio.transform.rowcol(wind_affine, x, y)
    return wind_speed[row, col]

# Modifica la funzione per estrarre anche la velocità media del vento per segmento
def extract_segments_elevation_and_wind(line, pixel_size):
    interpolated_points = interpolate_points(line, pixel_size)
    elevations = [get_elevation(p.x, p.y) for p in interpolated_points]
    wind_speeds = [get_wind_speed(p.x, p.y) for p in interpolated_points]
    
    elevation_differences = [j - i for i, j in zip(elevations[:-1], elevations[1:])]
    average_wind_speeds = [np.mean(wind_speeds[i:i+2]) for i in range(len(wind_speeds)-1)]  # Media tra i punti adiacenti

    # Crea segmenti e associa le differenze di elevazione e la media della velocità del vento
    segments = [LineString([interpolated_points[i], interpolated_points[i+1]]) for i in range(len(interpolated_points) - 1)]
    
    return pd.DataFrame({'geometry': segments, 'elevation_difference': elevation_differences, 'average_wind_speed': average_wind_speeds})

# Applica la funzione modificata al tuo GeoDataFrame
segments_with_wind_list = gdf['geometry'].apply(lambda geom: extract_segments_elevation_and_wind(geom, pixel_size))

# Concatena tutti i DataFrames in uno solo
segments_with_wind_df = pd.concat(segments_with_wind_list.tolist()).reset_index(drop=True)

# Crea un nuovo GeoDataFrame dai segmenti con informazioni aggiuntive sulla velocità del vento
segments_with_wind_gdf = gpd.GeoDataFrame(segments_with_wind_df, geometry='geometry')
segments_with_wind_gdf.crs = gdf.crs

In [23]:
segments_with_wind_gdf

,geometry,elevation_difference,average_wind_speed
0,"LINESTRING (564732.318 4928435.168, 564937.715...",32.419861,0.657223
1,"LINESTRING (564937.715 4928577.688, 565143.112...",-93.663788,0.428541
2,"LINESTRING (565143.112 4928720.209, 565348.509...",48.302917,0.200042
3,"LINESTRING (565348.509 4928862.729, 565553.906...",-45.006104,0.200210
4,"LINESTRING (565553.906 4929005.249, 565759.303...",-1.873901,0.200042
...,...,...,...
1134,"LINESTRING (750784.533 4891275.451, 750897.574...",46.122627,0.342288
1135,"LINESTRING (750897.574 4891052.467, 751010.614...",13.971512,0.342288
1136,"LINESTRING (751010.614 4890829.483, 751123.655...",-33.122108,0.573236
1137,"LINESTRING (751123.655 4890606.499, 751236.695...",31.066200,0.342689


In [27]:
def calcola_consumo_energetico(segmento, elevazione_differenza, velocità_vento_media):
    # Sostituisci i seguenti valori con quelli effettivi dalla formula del paper.
    # Esempio di parametri che potrebbero essere nella formula:
    costo_energetico_base = 1.0  # energia di base consumata per metro
    fattore_elevazione = 0.1  # energia addizionale consumata per metro di elevazione
    fattore_vento = 0.05  # energia addizionale consumata per unità di velocità del vento

    # Calcola la lunghezza del segmento
    lunghezza_segmento = segmento.length

    # Applica la formula dal paper scientifico
    consumo_energetico = (costo_energetico_base +
                          fattore_elevazione * elevazione_differenza +
                          fattore_vento * velocità_vento_media) * lunghezza_segmento

    return consumo_energetico

In [37]:
# Calcolo del consumo energetico per ogni segmento
segments_with_wind_gdf['consumo_energetico'] = segments_with_wind_gdf.apply(
    lambda row: calcola_consumo_energetico(row['geometry'], row['elevation_difference'], row['average_wind_speed']),
    axis=1
)

segments_with_wind_gdf['consumo_energetico_abs'] = segments_with_wind_gdf['consumo_energetico'].abs()

In [38]:
consumo_energetico_totale = segments_with_wind_gdf['consumo_energetico_abs'].sum()

In [39]:
consumo_energetico_totale

466644.47863828845

In [40]:
segments_with_wind_gdf

,geometry,elevation_difference,average_wind_speed,consumo_energetico,consumo_energetico_abs
0,"LINESTRING (564732.318 4928435.168, 564937.715...",32.419861,0.657223,1068.711807,1068.711807
1,"LINESTRING (564937.715 4928577.688, 565143.112...",-93.663788,0.428541,-2086.237939,2086.237939
2,"LINESTRING (565143.112 4928720.209, 565348.509...",48.302917,0.200042,1460.073458,1460.073458
3,"LINESTRING (565348.509 4928862.729, 565553.906...",-45.006104,0.200210,-872.649968,872.649968
4,"LINESTRING (565553.906 4929005.249, 565759.303...",-1.873901,0.200042,205.652987,205.652987
...,...,...,...,...,...
1134,"LINESTRING (750784.533 4891275.451, 750897.574...",46.122627,0.342288,1407.344286,1407.344286
1135,"LINESTRING (750897.574 4891052.467, 751010.614...",13.971512,0.342288,603.566400,603.566400
1136,"LINESTRING (751010.614 4890829.483, 751123.655...",-33.122108,0.573236,-570.887258,570.887258
1137,"LINESTRING (751123.655 4890606.499, 751236.695...",31.066200,0.342689,1030.938616,1030.938616
